In [1]:
"""For altering memory usage"""
from sage.interfaces.gap import get_gap_memory_pool_size, set_gap_memory_pool_size
n = get_gap_memory_pool_size()
set_gap_memory_pool_size(200*n)
n = get_gap_memory_pool_size()

In [10]:
"""List of indices of small groups by GAP library: [x,y]."""
small = []
for i in range(100,101):
    if len(gap.IdsOfAllSmallGroups(i+1)) < 150:
        small += gap.IdsOfAllSmallGroups(i+1)
    else:
        print(i+1)

In [11]:
"""For new problem with general subgroups."""
class induce():
    """Function to return list of class sizes in default order."""
    def __init__(self):
        pass

    """Returns class sizes up to conjugacy"""
    def class_sizes(self, G):
        group_order = G.order()
        reps = G.conjugacy_classes_representatives()
        class_sizes = []
        for g in reps:
            class_sizes.append(group_order / G.centralizer(g).order())
#         print(class_sizes)
#         orders = []
#         for rep in reps:
#             orders.append(rep.order())
#         print(orders)
        return class_sizes

    """Function to find all subgroups. Returns list of subgroups."""
    def subgroups(self, G):
        subgroups = G.conjugacy_classes_subgroups()
#         print(subgroups)
        return subgroups

    """Function to find induced trivial character of each subgroup. Returns a list of lists."""
    def get_induced(self, subgroups, G):
        ind = []
        for subgroup in subgroups:
            ind.append(subgroup.trivial_character().induct(G))
#         print(ind)
        return ind

    """Forms the matrices that we use to solve the simultaneous equation later. The last entry is the trivial representation."""
    def mat(self, induced):
        matrix = []
        for char in induced:
            matrix.append(list(char.values()))
        neg = []
        for i in range(len(matrix[0])):
            neg.append(-1)
        matrix[-1] = neg
#         for line in matrix:
#             print(line)
        return matrix

    """Check if a subgroup H of G is core-free"""
    def core_free(self, G, H):
        if H.order() == 1: 
            return True
        core = self.subgroups(H)[1:]
        for subsub in core:
            if subsub.is_normal(G) == True:
                return False
        return True

    """Find all subgroups of G containing H (just the conjugates)"""
    def subgroups_containing(self, G, H, subgroups):
        subgroups_list = [] 
        h = H.order()
        for subgroup in subgroups:
            if subgroup.order() > h :
                glist = G.list()
                sub = False
                while sub == False and len(glist) != 0:
                    g = glist.pop()
                    hh = subgroup.conjugate(g)
                    sub = H.is_subgroup(hh)
                if sub == True:
                    subgroups_list.append(subgroup)
        return subgroups_list

    """Test if there exists non-negative solutions. Prints the id of group that works and the ci, prints fail otherwise."""
    def test(self, G):
        a = self.class_sizes(G)
        b = self.subgroups(G)
        zeros = [0]*len(a)
        ret_list = ""
        for i in range(len(b)):
            cor = self.core_free(G, b[i])
            if cor == True:
                try:
                    b_i = self.subgroups_containing(G, b[i], b)
                    c = self.get_induced(b_i, G)
                    d_i = self.mat(c)
                    e = Matrix(d_i).T
                    temp = b[i].trivial_character().induct(G).values()
#                     print(temp)
                    Y = vector(temp)
                    p = MixedIntegerLinearProgram(solver='GLPK')
                    w = p.new_variable(nonnegative=True)
                    p.add_constraint(e*w == Y)
                    p.solve()
                    out = []
                    for j, v in (p.get_values(w).items()):
                        out.append('w_%s = %s' % (j, v))
                        """Prints subgroup and subgroup order."""
#                         if round(v,6) != 0:
#                             print(e.T[j])
#                             print(b_i[j], b_i[j].order())
                    """Prints the solution and SmallGroup Id"""
                    ret_list += str(gap.IdSmallGroup(b[i])[1]) + "," + str(gap.IdSmallGroup(b[i])[2])+"  "
#                     print(str(gap.IdSmallGroup(G)[1]) + "," + str(gap.IdSmallGroup(G)[2]) +"    "+ str(gap.IdSmallGroup(b[i])[1]) + "," + str(gap.IdSmallGroup(b[i])[2]))
#                     print(e.T)
#                     print(b[i], b[i].order())
#                     print(out)
                except:
                    pass
#                     print('fail')
        return ret_list

In [ ]:
"""Main function
If group is in id form.
https://www.gap-system.org/Manuals/pkg/SmallGrp-1.3/doc/chap1.html
This works on CoCalc/non-windows system with gap_packages library installed."""
if __name__ == "__main__":
    inducer = induce()
    groups = small
    out_a = []
    out_b = []
    for groupid in groups:
        G = gap.SmallGroup(groupid)
        ret = inducer.test(PermutationGroup(gap_group = G.AsPermGroup()))
        if len(ret) > 0:
            out_a.append(groupid)
            out_b.append(ret)
#         try:
#             inducer.test(PermutationGroup(gap_group = G.AsPermGroup()))
#         except:
#             pass
    for a in out_a:
        print(str(a[1])+","+str(a[2]))
    for b in out_b:
        print(b)
    """If group is in SAGE form.
    https://doc.sagemath.org/html/en/thematic_tutorials/group_theory.html#groups-of-small-order-as-permutation-groups"""
#     C5 = CyclicPermutationGroup(5)
#     G = C5.holomorph()
#     C2 = CyclicPermutationGroup(2)
#     groups = [direct_product_permgroups([C2,G])]
#     for group in groups:
#         try:
#             result = inducer.test(group)
#         except:
#             pass

102,3    1,1
104,6    1,1
104,12    2,1
104,13    1,1
106,1    1,1
108,4    1,1
108,22    1,1
108,25    3,1
108,26    3,1
108,27    1,1
108,37    2,1
108,41    1,1
108,44    1,1
110,1    1,1
110,1    2,1
110,1    5,1
110,5    1,1
111,1    1,1
112,6    1,1
112,29    1,1
112,42    1,1
114,1    1,1
114,1    2,1
114,1    3,1
114,5    1,1
116,3    1,1
116,3    2,1
116,4    1,1
117,3    1,1
118,1    1,1
120,28    1,1
120,34    1,1
120,38    2,1
120,41    2,1
120,46    1,1
121,2    1,1
122,1    1,1
124,3    1,1
125,3    1,1
125,5    1,1
126,5    1,1
126,7    2,1
126,9    3,1
126,15    1,1
129,1    1,1
130,3    1,1
132,9    1,1
134,1    1,1
136,6    1,1
136,12    1,1
136,12    2,1
136,12    4,1
136,13    2,1
136,14    1,1
138,3    1,1
140,6    2,1
140,10    1,1
142,1    1,1
146,1    1,1
147,1    1,1
147,4    1,1
147,4    3,1
147,5    1,1
148,3    1,1
148,3    2,1
148,4    1,1
150,3    1,1
150,5    5,1
150,6    1,1
150,6    2,1
150,6    3,1
150,12    1,1
152,5    1,1
152,11    1,1
154,3    1,1
